In [ ]:
from tkinter import Tk, filedialog
from ipywidgets import *
from IPython.display import clear_output, display

In [ ]:
class FileCollector:
    
    def __init__(self):
        self.file_list = None
        
        self.select_file_button = widgets.Button(
            description='Select File',
            tooltip='Find and select a file to be read'
        )        
        self.select_file_button.on_click(self.select_files)
        display(self.select_file_button)

    def select_files(self, b):
        root = Tk()
        root.withdraw()                                        # Hide the main window.
        root.call('wm', 'attributes', '.', '-topmost', True)
        self.file_list = filedialog.askopenfilename(multiple=True) 
    
    def get_files(self):
        return self.file_list

# Create array

In [ ]:
array_files = FileCollector()

In [ ]:
array_paths = array_files.get_files()
array_paths[0]

In [ ]:
import tifffile
tiff_array = tifffile.TiffFile(array_paths[0]).asarray()
print(tiff_array.shape)
print(tiff_array)

In [ ]:
array_metadata={"dataset" : "sandbox", "element":{"symbol": "Co", "edge": "K"}}
print(array_metadata)

# Create DataFrames

In [ ]:
dataframe_files = FileCollector()

In [ ]:
dataframe_paths = dataframe_files.get_files()
dataframe_paths[0]

In [ ]:
import json
import pandas

In [ ]:
with open(dataframe_paths[0]) as json_file:
    file_data = json.load(json_file)
    dataframe_metadata = {}
    data = {}
    for item in file_data:
        if "Energy" in item:
            data = item.copy()
        else:
            if "element" in item:
                element = {"element":{"symbol": item["element"], "edge": item["edge"]}}
                item.pop("element")
                item.pop("edge")
                item.update(element)
            dataframe_metadata.update(item)
    
    json_dataframe = pandas.DataFrame(data)
    dataframe_metadata.update({"dataset" : "sandbox"})

print(json_dataframe.shape)
json_dataframe.head()

In [ ]:
dataframe_metadata

# Start Tiled client

In [ ]:
from tiled.client import from_uri

In [ ]:
client = from_uri("https://aimm.lbl.gov/api")

In [ ]:
client.login()

In [ ]:
client

# Write array

In [ ]:
array_key = client['uid'].write_array(tiff_array, array_metadata)
array_key

# Write dataframe

In [ ]:
df_key = client["uid"].write_dataframe(json_dataframe, dataframe_metadata)
df_key

# Validate write/upload

In [ ]:
#from databroker.experimental.queries import RawMongo
from aimmdb.tree import RawMongoQuery as RawMongo

In [ ]:
results = client["dataset"]["sandbox"].search(RawMongo({"metadata.element.symbol":"Co"})) # Search for all element that meet the search criteria
list(results)

In [ ]:
array_result = results.values_indexer[0].read() # Get first item of all results
results.values_indexer[0].metadata

In [ ]:
array_result

In [ ]:
dataframe_result = results.values_indexer[1].read()
results.values_indexer[1].metadata

In [ ]:
dataframe_result